<a href="https://colab.research.google.com/github/CS23M005/Assignment2_PartA/blob/main/CS23M005_A2_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
device

device(type='cuda')

In [3]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-03 18:10:30--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.162.207, 74.125.134.207, 74.125.139.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.162.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   120MB/s    in 28s     

2024-04-03 18:10:58 (129 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [4]:
!rm nature_12K.zip

In [5]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets

transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[7999,2000])

train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size =32,shuffle = True,num_workers=2,pin_memory=True)

test_data = torch.utils.data.DataLoader(
    datasets.ImageFolder(root='inaturalist_12K/val',transform=transform),batch_size = 32,shuffle = True,num_workers=2,pin_memory=True
)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, input_channel, output_size, num_filters, filter_size, activation_fun, filter_config, stride, neurons):
        super(ConvNet,self).__init__()
        self.k = filter_size
        m = self.getM()
        self.s = stride
        w, h = 256,256

        self.conv1 = nn.Conv2d(input_channel, m[0], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool1 = nn.MaxPool2d(self.k,self.s)
        w,h = self.getWH(w,h)
        self.conv2 = nn.Conv2d(m[0], m[1], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool2 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv3 = nn.Conv2d(m[1], m[2], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool3 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv4 = nn.Conv2d(m[2], m[3], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool4 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv5 = nn.Conv2d(m[3], m[4], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool5 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.fc1 = nn.Linear(m[4]*w*h, neurons)
        self.fc2 = nn.Linear(neurons,10)



    def getWH(self, w, h):
      return (math.floor(((w-self.k)/self.s)+1),math.floor(((w-self.k)/self.s)+1))

    def getM(self):
        m = []
        if(filter_config == "double"):
          for i in range(1,6):
            m.append((int)(2**(i)*num_filters))
        elif(filter_config == "same"):
          for i in range(1,6):
            m.append(int(num_filters))
        elif(filter_config == "half"):
          for i in range(1,6):
            m.append((int)(num_filters/(2**(i))))
        return m

    def getActivation_fn(self):
        if(activation_fun == "relu"):
            activation_fn = F.relu
        elif(activation_fun == "gelu"):
            activation_fn = F.gelu
        elif(activation_fun == "silu"):
            activation_fn = F.silu
        elif(activation_fun=="mish"):
            activation_fn = F.mish
        return activation_fn

    def forward(self, x):
        activation_fn = self.getActivation_fn()
        x = activation_fn(self.conv1(x))
        x = self.pool1(x)
        x = activation_fn(self.conv2(x))
        x = self.pool2(x)
        x = activation_fn(self.conv3(x))
        x = self.pool3(x)
        x = activation_fn(self.conv4(x))
        x = self.pool4(x)
        x = activation_fn(self.conv5(x))
        x = self.pool5(x)
        x = x.reshape(x.shape[0], -1)
        x = activation_fn(self.fc1(x))
        x = self.fc2(x)
        return x



In [7]:
def getOptim(model,optim_name, learning_rate):
  if(optim_name == 'sgd'):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
  elif(optim_name == 'adam'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  else:
    optimizer = optim.NAdam(model.parameters(), lr=learning_rate)
  return optimizer


In [8]:
def check_accuracy(loader,model,criterion,batchSize):
    num_correct = 0
    num_loss = 0
    total = 0
    num_samples = 0
    total_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batchSize
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples)*100 , total_loss

In [9]:
def train_cnn(input_channel, output_size, num_filters, filter_size, activation_fun, filter_config,
              stride, neurons,optim_name,batchSize,dropOut,num_epochs,learning_rate):

    transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

    train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)

    train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])

    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)


    model = ConvNet(input_channel, output_size, num_filters, filter_size, activation_fun, filter_config, stride, neurons).to(device)
    optimizer = getOptim(model,optim_name, learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            data = data.to(device=device)
            targets = targets.to(device=device)
            scores = model(data)
            loss = criterion(scores,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_accuracy,train_loss = check_accuracy(train_loader, model,criterion,batchSize)
        validation_accuracy,validation_loss = check_accuracy(val_loader, model,criterion,batchSize)
        print(f"train_accuracy:{train_accuracy:.4f},train_loss:{train_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})

    #wandb.log({'train_accuracy':train_accuracy})


In [10]:
!pip install wandb
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
optim_name = 'adam'
batchSize=32
dropOut = 0.1
num_epochs = 5
learning_rate = 1e-3
input_channel=3
output_size=10
num_filters=16
filter_size=3
activation_fun = "relu"
filter_config = "same"
stride = 1


In [ ]:
def main_fun():
    wandb.init(project ='Assignment2_PartA')
    params = wandb.config
    with wandb.init(project = 'Assignment2_PartA', name='neurons'+str(params.neurons)+'filterSize'+str(params.filter_size)+'activFun'+params.activation_fun) as run:
        train_cnn(input_channel, output_size, params.num_filters, params.filter_size, params.activation_fun, params.filter_config,
              stride, params.neurons,params.optim_name, params.batchSize, params.dropOut, params.num_epochs, params.learning_rate)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'cs23m005',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'neurons':{'values':[64,128,256]},
        'num_filters' : {'values' : [16,32]},
        'filter_size':{'values' : [3,5,11]},
        'activation_fun' :{'values':['relu','gelu','silu','mish']},
        'filter_config' : {'values':['same','double','half']},
        'optim_name' :{'values':['sgd','adam']},
        'batchSize' : {'values':[32,64]},
        'dropOut' :{'values':[0.1]},
        'num_epochs':{'values':[5,10]},
        'learning_rate' :{'values':[1e-3,1e-4]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'Assignment2_PartA')
wandb.agent(sweepId,function =main_fun,count = 20)
wandb.finish()

Create sweep with ID: dgtgu0yd
Sweep URL: https://wandb.ai/cs23m005/Assignment2_partA/sweeps/dgtgu0yd


wandb: Agent Starting Run: 1nbn52i9 with config:
wandb: 	activation_fun: silu
wandb: 	batchSize: 32
wandb: 	dropOut: 0.1
wandb: 	filter_config: same
wandb: 	filter_size: 11
wandb: 	learning_rate: 0.0001
wandb: 	neurons: 256
wandb: 	num_epochs: 5
wandb: 	num_filters: 16
wandb: 	optim_name: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


100%|██████████| 250/250 [01:37<00:00,  2.57it/s]
